

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [1]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26
device = 'cuda' if torch.cuda.is_available() else 'cpu'


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [2]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.gru = torch.nn.GRU(
        input_size=1+N,
        hidden_size=hidden_size,
        num_layers=2,
        batch_first=True
    )
    self.fc = torch.nn.Linear(hidden_size, 1+N)

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    x = self.gru(x)[0]
    return self.fc(x)

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    seq = torch.tensor([ord(i) - ord('a') + 1 for i in s])
    one_hot = idx_to_onehot(seq).unsqueeze(0).to(device)
    pred = self.forward(one_hot)
    pred = torch.argmax(pred, 2)
    out = ''
    for c in pred[0]:
        if c == 0:
            out += ' '
        else:
            out += chr(ord('a') + c - 1)
    return out

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [3]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
      correct += int(c1 == c2)
    total += len(s) - DELAY

  return correct / total

In [4]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    model.to(device)
    correct = 0
    total = 0
    for inp, target in dataloader:
        inp = idx_to_onehot(inp).float().to(device)
        target_onehot = idx_to_onehot(target).float().to(device)
        # print(inp)
        # print(target_onehot)
        out = model(inp)
        # print(out)
        loss = criterion(out[:, DELAY:, :], target_onehot[:, DELAY:, :])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred = torch.argmax(out, 2).detach().cpu()
        correct += torch.sum(pred[:, DELAY:] == target[:, DELAY:])
        total += target.shape[0]*(target.shape[1] - DELAY)
    
    return correct / total

In [5]:
import time
start_time = time.time()

# TODO
model = GRUMemory(64).to(device)
train_loader = torch.utils.data.DataLoader(ds, batch_size=32)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 1

for e in range(epochs):
    accuracy = train(model, train_loader, optimizer, criterion)
    print('Epoch ', e+1, accuracy)


end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

Epoch  1 tensor(0.9461)
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [6]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [7]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    #TODO
    self.max_delay = max_delay
    self.hidden_size = hidden_size
    self.linear = torch.nn.Linear(1, hidden_size)
    self.gru = torch.nn.GRU(
        input_size=1+N,
        hidden_size=hidden_size,
        num_layers=3,
        dropout=0.2,
        batch_first=True
    )
    self.fc = torch.nn.Linear(hidden_size, 1+N)

  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

    # TODO
    # hidden0 = torch.mul(torch.ones(x.shape[0], self.hidden_size).to(device), delays.view(-1, 1)).unsqueeze(0)
    hidden0 = self.linear(delays.view(-1, 1)).unsqueeze(0)
    hidden = torch.cat([hidden0, hidden0, hidden0], 0)
    x = self.gru(x, hidden.to(device))[0]
    return self.fc(x)

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    seq = torch.tensor([ord(i) - ord('a') + 1 for i in s])
    one_hot = idx_to_onehot(seq).unsqueeze(0).to(device)
    pred = self.forward(one_hot, torch.tensor([delay]).float().to(device))
    pred = torch.argmax(pred, 2)
    out = ''
    for c in pred[0]:
        if c == 0:
            out += ' '
        else:
            out += chr(ord('a') + c - 1)
    return out

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [8]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [18]:
def train_delay(model, dataloader, optimizer, criterion, scheduler):
    model.train()
    model.to(device)
    correct = 0
    total = 0
    for inp, d, target in dataloader:
        inp = idx_to_onehot(inp).float().to(device)
        d = d.float().to(device)
        target_onehot = idx_to_onehot(target).float().to(device)
        # print(inp)
        # print(target_onehot)
        out = model(inp, d)
        # print(out)
        loss = criterion(out[:, int(d):, :], target_onehot[:, int(d):, :])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        pred = torch.argmax(out, 2).detach().cpu()
        correct += torch.sum(pred[:, int(d):] == target[:, int(d):])
        total += target.shape[0]*(target.shape[1] - int(d))
    
    return correct / total

In [19]:
import time
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20

# TODO: implement model training here.
model = VariableDelayGRUMemory(64, MAX_DELAY).to(device)
train_set = VariableDelayEchoDataset(MAX_DELAY, SEQ_LENGTH, 5000)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)
epochs = 30
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 0.001, epochs=epochs, steps_per_epoch=5000)

for e in range(epochs):
    accuracy = train_delay(model, train_loader, optimizer, criterion, scheduler)
    # scheduler.step()
    print('Epoch ', e+1, accuracy)

end_time = time.time()
print(test_variable_delay_model(model))
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

Epoch  1 tensor(0.0518)
Epoch  2 tensor(0.0702)
Epoch  3 tensor(0.1077)
Epoch  4 tensor(0.2192)
Epoch  5 tensor(0.3642)
Epoch  6 tensor(0.4770)
Epoch  7 tensor(0.5554)
Epoch  8 tensor(0.6108)
Epoch  9 tensor(0.6630)
Epoch  10 tensor(0.7142)
Epoch  11 tensor(0.7578)
Epoch  12 tensor(0.7828)
Epoch  13 tensor(0.8133)
Epoch  14 tensor(0.8289)
Epoch  15 tensor(0.8481)
Epoch  16 tensor(0.8658)
Epoch  17 tensor(0.8820)
Epoch  18 tensor(0.8879)
Epoch  19 tensor(0.9021)
Epoch  20 tensor(0.9160)
Epoch  21 tensor(0.9263)
Epoch  22 tensor(0.9348)
Epoch  23 tensor(0.9433)
Epoch  24 tensor(0.9504)
Epoch  25 tensor(0.9600)
Epoch  26 tensor(0.9650)
Epoch  27 tensor(0.9698)
Epoch  28 tensor(0.9722)
Epoch  29 tensor(0.9737)
Epoch  30 tensor(0.9738)
0.9765644477686363


AssertionError: ignored